In [1]:
import pandas as pd

# Total violations

Totaling deficiencies from operation listings.

The first table shows daycares with at least 100 violations of any severity as of Feb. 1.

In [2]:
dat_operations = pd.read_csv(
    '../csv/dfps-2018-02-01/operations.csv'
).query(
    'OPERATION_TYPE != "Child Placing Agency" & OPERATION_TYPE != "General Residential Operation"'
)[[
    'OPERATION_ID',
    'OPERATION_TYPE',
    'OPERATION_NUMBER',
    'OPERATION_NAME',
    'LOCATION_ADDRESS',
    'COUNTY',
    'DEFICIENCY_HIGH',
    'DEFICIENCY_MEDIUM_HIGH',
    'DEFICIENCY_MEDIUM',
    'DEFICIENCY_MEDIUM_LOW',
    'DEFICIENCY_LOW'
]].set_index([
    'OPERATION_ID',
    'OPERATION_TYPE',
    'OPERATION_NUMBER',
    'OPERATION_NAME',
    'LOCATION_ADDRESS',
    'COUNTY'
]).assign(
    total_violations = lambda x: x.DEFICIENCY_HIGH +
        x.DEFICIENCY_MEDIUM_HIGH +
        x.DEFICIENCY_MEDIUM +
        x.DEFICIENCY_MEDIUM_LOW +
        x.DEFICIENCY_LOW
)

print('{} daycares with at least 100 violations'.format(
    len(dat_operations.sort_values('total_violations', ascending=False).query(
        'total_violations >= 100'
    ).index)
))

dat_operations.sort_values('total_violations', ascending=False).query(
    'total_violations >= 100'
)[[
    'total_violations'
]].to_clipboard()

7 daycares with at least 100 violations


In [3]:
dat_operations = pd.read_csv(
    '../csv/dfps-2018-02-01/operations.csv'
).query(
    'OPERATION_TYPE != "Child Placing Agency" & OPERATION_TYPE != "General Residential Operation"'
)[[
    'OPERATION_ID',
    'OPERATION_TYPE',
    'OPERATION_NUMBER',
    'OPERATION_NAME',
    'LOCATION_ADDRESS',
    'COUNTY',
    'DEFICIENCY_HIGH',
    'DEFICIENCY_MEDIUM_HIGH',
    'DEFICIENCY_MEDIUM',
    'DEFICIENCY_MEDIUM_LOW',
    'DEFICIENCY_LOW'
]].set_index([
    'OPERATION_ID',
    'OPERATION_TYPE',
    'OPERATION_NUMBER',
    'OPERATION_NAME',
    'LOCATION_ADDRESS',
    'COUNTY'
]).assign(
    total_violations = lambda x: x.DEFICIENCY_HIGH +
        x.DEFICIENCY_MEDIUM_HIGH +
        x.DEFICIENCY_MEDIUM +
        x.DEFICIENCY_MEDIUM_LOW +
        x.DEFICIENCY_LOW
)

print('{} daycares with at least 50 violations'.format(
    len(dat_operations.sort_values('total_violations', ascending=False).query(
        'total_violations >= 50'
    ).index)
))

dat_operations.sort_values(
    'total_violations', ascending=False
).to_clipboard()

116 daycares with at least 50 violations


The table below queries the live database and shows daycares with at least 100 violations of any severity.

In [4]:
dat_live_operations = pd.read_csv(
    'https://data.texas.gov/api/views/bc5r-88dy/rows.csv?accessType=DOWNLOAD'
).query(
    'OPERATION_TYPE != "Child Placing Agency" & OPERATION_TYPE != "General Residential Operation"'
)[[
    'OPERATION_ID',
    'OPERATION_TYPE',
    'OPERATION_NUMBER',
    'OPERATION_NAME',
    'LOCATION_ADDRESS',
    'COUNTY',
    'DEFICIENCY_HIGH',
    'DEFICIENCY_MEDIUM_HIGH',
    'DEFICIENCY_MEDIUM',
    'DEFICIENCY_MEDIUM_LOW',
    'DEFICIENCY_LOW'
]].set_index([
    'OPERATION_ID',
    'OPERATION_TYPE',
    'OPERATION_NUMBER',
    'OPERATION_NAME',
    'LOCATION_ADDRESS',
    'COUNTY'
]).assign(
    total_violations = lambda x: x.DEFICIENCY_HIGH +
        x.DEFICIENCY_MEDIUM_HIGH +
        x.DEFICIENCY_MEDIUM +
        x.DEFICIENCY_MEDIUM_LOW +
        x.DEFICIENCY_LOW
)

print('{} daycares with at least 100 violations'.format(
    len(dat_live_operations.sort_values('total_violations', ascending=False).query(
        'total_violations >= 100'
    ).index)
))

dat_live_operations.sort_values('total_violations', ascending=False).query(
    'total_violations >= 100'
)[[
    'total_violations'
]].to_clipboard()

26 daycares with at least 100 violations


In [5]:
print('{} daycares with at least 50 violations'.format(
    len(dat_live_operations.sort_values('total_violations', ascending=False).query(
        'total_violations >= 50'
    ).index)
))

263 daycares with at least 50 violations


Table below: of the daycares with at least 100 violations as of Feb. 1, which are still listed?

In [6]:
dat_operations.assign(
    still_listed = lambda x: x.index.isin(dat_live_operations.index)
).query(
    'total_violations >= 100'
)[['still_listed']].sort_values(
    'still_listed'
)

,,,,,,still_listed
OPERATION_ID,OPERATION_TYPE,OPERATION_NUMBER,OPERATION_NAME,LOCATION_ADDRESS,COUNTY,
1143847,Licensed Center,1538844,Rugratz Learning Academy,930 W MAIN ST DENISON TX- 75020 3348,GRAYSON,False
1221905,Licensed Center,1606396,Learning Palace 2,9647 SOUTHWEST FWY HOUSTON TX- 77074 1332,HARRIS,False
1153391,Licensed Center,1545543,All Star Kidz Learning Center,2703 W EXPWY 83 MISSION TX- 78572 7048,HIDALGO,True
1247270,Licensed Center,1627850,"B & 3Js Big Kidz Plaz, Inc",1201 W CAMP WISDOM RD STE 320 DALLAS TX- 75232 3446,DALLAS,True
314262,Licensed Center,871757,Little Feet & Learning Day Care Center,1001 BRISTOL RD STE A LAREDO TX- 78045 8536,WEBB,True
1219078,Licensed Center,1603276,Lil Dragon Daycare,8502 PEARSALL RD SAN ANTONIO TX- 78252 2662,BEXAR,True
1241212,Licensed Center,1623261,Early Learning Center,10851 SCARSDALE BLVD STE 700 HOUSTON TX- 77089 5742,HARRIS,True


### Test

Compare operation violation totals to non-compliance table violation totals

In [7]:
import unittest

count_non_compliance = len(
    pd.read_csv(
        '../csv/dfps-2018-02-01/non_compliance.csv'
    ).query('OPERATION_ID == {}'.format(
        dat_operations.sort_values('total_violations',ascending=False
        ).reset_index().OPERATION_ID.iloc[0]
        )
   ).index
)

count_operations = dat_operations.sort_values(
    'total_violations',
    ascending=False
).iloc[0].total_violations

if count_operations == count_non_compliance:
    print('Test passed')
else:
    raise Exception('{} != {}'.format(
        count_operations,
        count_non_compliance
    ))

Test passed


### Violation distribution

In [8]:
print(dat_operations.total_violations.describe())

pd.DataFrame(
    dat_operations.total_violations.quantile((
        .8, .9, .99
    )).rename(
        lambda x: '{:.1%}'.format(x)
    )
)

count    15070.000000
mean         7.014798
std          9.831177
min          0.000000
25%          1.000000
50%          4.000000
75%          9.000000
max        214.000000
Name: total_violations, dtype: float64


,total_violations
80.0%,11.0
90.0%,17.0
99.0%,46.0


In [9]:
print(dat_live_operations.total_violations.describe())

pd.DataFrame(
    dat_live_operations.total_violations.quantile((
        .8, .9, .99
    )).rename(
        lambda x: '{:.1%}'.format(x)
    )
)

count    14998.00000
mean         9.24690
std         12.61579
min          0.00000
25%          2.00000
50%          5.00000
75%         12.00000
max        172.00000
Name: total_violations, dtype: float64


,total_violations
80.0%,14.0
90.0%,23.0
99.0%,59.0


In [11]:
dat_live_operations.total_violations.median()

5.0